# ![HELP LOGO](https://avatars1.githubusercontent.com/u/7880370?s=100&v=4>) Spitzer Spuds MIPS map homogenisation


These MIPS maps have ben produced by the Spitzer SPUDS project. This code takes the individual image files and adds them into a single multi-layered fits files to have the homogeneous HELP format.  This code was adapted from code used for SPIRE homogenisaton (Yannick Roehlly) and in turn adapted for PACS data (Peter Hurely) and further adapted for SEIP data (Seb Oliver). 

## Caveats


------------------------------------------------------------------

<h2> Guide</h2>
<h3>Steps: </h3>
* Obtain list of fields from sub-folders
for each field:
* read in coverage, image and noise
* attach meta data
* create PSF extension
* Save out

<h3>Things you may wish to edit</h3>
* You'll need to change the folder path in the 2nd code block to point you where the files are in your system
* Add in a specifc field (or switch to wild chracter) in the path to pick field(s)
* change the 1.mosaic pattern to 2.mosaic for MIPS 70 micron or 3.mosaic for MIPS 160.
* read in more of the available files and add as additional extensions

<h2> Getting Original Data</h2>
<ul>
<li>wget https://irsa.ipac.caltech.edu/data/SPITZER/SpUDS/images/mips/mips_mosaic.fits data/
<li>wget https://irsa.ipac.caltech.edu/data/SPITZER/SpUDS/images/mips/mips_mosaic_cov.fits data/
<li>wget https://irsa.ipac.caltech.edu/data/SPITZER/SpUDS/images/mips/mips_mosaic_unc.fits data/
    </ul>

# Importing libraries

In [1]:
from datetime import datetime
from itertools import product
import glob

import numpy as np
import astropy as astropy

from astropy.io import fits
from astropy.table import Table
from astropy.table import Column
from astropy import wcs
VERSION = "0.9"

# This is where you set up the field and folders to work on

In [2]:
field='XMM-LSS' 

mainfolder='../../dmu17/dmu17_XMM-LSS/data/'
file=mainfolder+'mips_mosaic.fits'
noise_file=mainfolder+'mips_mosaic_unc.fits'
cov_file=mainfolder+'mips_mosaic_cov.fits'


# Performing the homogenisation
i.e. merging component files into a single multi-extension file and adding some keywords
traps errors so that it can continue, e.g. if a file is missing
skips over files if they've already been logged as "downloaded" (this could be improved to be an option

In [3]:
# Load in image map

ext=0

image_map=fits.open(file)
image_hdu = fits.ImageHDU(header=image_map[ext].header,
                                data=image_map[ext].data)
image_hdu.header['EXTNAME'] = "IMAGE"
 
# Load in uncertainty map
noise_map=fits.open(noise_file)
noise_hdu = fits.ImageHDU(header=noise_map[ext].header,
                                data=noise_map[ext].data)
noise_hdu.header['EXTNAME'] = "ERROR"
 
# we'll use the uncertainty map to generate a MOC
#good=~np.isnan(noise_map[ext].data)
#w = wcs.WCS(noise_map[ext].header)     
#m=create_MOC_from_map(good,w)
#total_moc=total_moc+m
        
#intersect=m.intersection(help_moc)
#print('Area',repr(intersect.area))
#        
#m.write(file.replace('mosaic','moc'), overwrite=True)
        
            # Load in coverage map
cov_map=fits.open(cov_file)
cov_hdu = fits.ImageHDU(header=cov_map[ext].header,
                                data=cov_map[ext].data)
cov_hdu.header['EXTNAME'] = "COVERAGE"

#Create primary header
primary_hdu = fits.PrimaryHDU()
primary_hdu.header.append((
            "CREATOR", "Herschel Extragalactic Legacy Project"
        ))
primary_hdu.header.append((
            "TIMESYS", "UTC", "All dates are in UTC time"
        ))
primary_hdu.header.append((
            "DATE", datetime.now().replace(microsecond=0).isoformat(),
            "Date of file creation"
        ))
primary_hdu.header.append((
        "VERSION", VERSION, "HELP product version"
        ))
primary_hdu.header.append((
        "TELESCOP", "Spitzer", "Name of the telescope"
        ))
primary_hdu.header.append((
        "INSTRUME", "MIPS", "Name of the instrument"
        ))
primary_hdu.header.append((
        "FILTER", "MIPS-24", "Name of the filter"
        ))
primary_hdu.header.append((
        "FIELD", field, "Name of the HELP field"
        ))


#Add empty extension for PSF
psf_hdu = fits.ImageHDU()
psf_hdu.header['EXTNAME'] = "PSF"
psf_hdu.header.add_comment("The PSF is not available.")

hdu_list = fits.HDUList([primary_hdu, image_hdu, noise_hdu, cov_hdu, psf_hdu])
hdu_list.writeto(file.replace('mosaic','help'),
                 checksum=True, overwrite=True)



-------------------------------------------------------------------------------

 ![HELP LOGO](https://avatars1.githubusercontent.com/u/7880370?s=75&v=4)

**Authors**:  [Seb Oliver](http://www.sussex.ac.uk/profiles/91548)

 
For a full description of the database and how it is organised in to `dmu_products` please the top level [readme](../readme.md).
 
The Herschel Extragalactic Legacy Project, ([HELP](http://herschel.sussex.ac.uk/)), is a [European Commission Research Executive Agency](https://ec.europa.eu/info/departments/research-executive-agency_en)
funded project under the SP1-Cooperation, Collaborative project, Small or medium-scale focused research project, FP7-SPACE-2013-1 scheme, Grant Agreement
Number 607254.

[Acknowledgements](http://herschel.sussex.ac.uk/acknowledgements)


Seb Oliver 6-May-2018